In [ ]:
from keras import layers, models
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## MNIST database

The MNIST (Modified National Institute of Standards and Technology) database is a public dataset of handwritten digits:

- There are 60,000 training images and 10,000 testing images in the dataset.
- Digits are normalized to fit in a 28x28 pixel bounding box.
- Each grayscale image pixel is represented by an 8-bit integer value between 0 (black) and 255 (white).
- Each image is annotated with the digit shown in the image. This annotation / label is called ground truth.

The figure below shows the first 20 training images as well as the respective ground truth labels.

![image.png](figures/mnist.png)

In [ ]:
# functions to load and visualize the data

def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # normalize images
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    # one-hot encode labels
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)

    return (x_train, y_train), (x_test, y_test)

def visualize_mnist(n):
    """For each digit, show the first n images from the training dataset.
    """
    assert n > 0
    (x, y), (_, _) = tf.keras.datasets.mnist.load_data()

    cols = 10
    rows = n

    plt.figure(figsize=(12, rows*1.5))
    for i in range(10):
        idx = y == i
        for j in range(n):
            plt.subplot(rows, cols, i+10*j+1)
            plt.imshow(x[idx][j], cmap='gray')
            plt.xlabel(i)
            plt.xticks([])
            plt.yticks([])
    plt.show()

In [ ]:
# load and preprocess data
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
# visualize the first 50 images from the training set
visualize_mnist(5)

## Neural Network Training

The neural nets (`keras.Model` instances) are trained on 90% of the training data.
The remaining 10%, i.e., 6,000 pairs of images and labels, are reserved for validation.

During training, we monitor both the loss of the training and the validation split over the training iterations (epochs).
If the validation loss does not decrease for more than three consecutive epochs, we end the training (*early stopping*).

In [ ]:
def train_model(model, patience=3):
    """Train the model as long as the validation loss is decreasing
    and return the history of the training process.
    """

    # stops training when a monitored metric (here validation loss) has stopped improving
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience)

    hist = model.fit(
        x_train, 
        y_train,
        shuffle=True,
        epochs=100,
        batch_size=512, 
        validation_split=0.1, 
        callbacks=[early_stopping],
    )
    return hist

def plot_history(hist):
    """Plot the training and validation loss and accuracy over the epochs."""

    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    # plot the training and validation accuracy in left subplot
    plt.plot(hist.history['accuracy'], label='training')
    plt.plot(hist.history['val_accuracy'], label='validation')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy [%]')
    plt.legend()

    plt.subplot(1, 2, 2)
    # plot the training and validation loss on logarithmic scale in right subplot
    plt.plot(hist.history['loss'], label='training')
    plt.plot(hist.history['val_loss'], label='validation')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss [-]')
    plt.legend()
    plt.show()

### Feedforward Network for Classification

We define the following fully-connected feedforward network architecture to classify the handwritten digit images:

1. Input Layer: 2D images represented as 28x28 matrix
2. Flatten Layer: Transforms the 2D images (28x28 pixels) into a 1D column vector of size 784=28^2
3. Hidden fully-connected layer with 128 neurons and ReLU activation
4. Output layer, fully connected with 10 neurons and Softmax activation. Models a probability distribution over the available digits 0-9.

Since our goal is multi-class classification, we choose the categorical cross-entropy as our loss function.
Moreover, we instruct the model to monitor the classification accuracy as an additional metric.

In [ ]:
# define the model
simple_ffn = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    # output layer: number of units is the number of classes
    tf.keras.layers.Dense(units=10, activation='softmax')
])
simple_ffn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
simple_ffn.summary()

In [ ]:
simple_ffn_hist = train_model(simple_ffn)

In [ ]:
plot_history(simple_ffn_hist)

### Hyperparamters

Beside the trainable parameters (i.e. weights and biases in the fully-connected layers) there are a number of hyperparameters which affect the overall architecture, size and training scheme of the neural network.
While trainable parameters are optimized with gradient decent to minimize the specified loss function, hyperparameters are design choices made by the programmer.
However, the choice of hyperparameters has a significant impact on the model's performance and thus needs to be optimized as well.
Such a hyperparameter optimization is steered by the model's performance on validation data and can be done manually or automatically.

Typical hyperparameters are:

- depth: number of hidden layers
- width: number of units (neurons) per (hidden) layer
- type of activation function
- optimizer
- regularization
- number of iterations (epochs): we have already optimized this parameter with early stopping

In the following, we will manually investigate the impact of the depth and width on the classification performance.

In [ ]:
# deep feedforward neural network
deep_ffn = models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.003)),
    tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.003)),
    tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.003)),
    # output layer: number of units is the number of classes
    tf.keras.layers.Dense(units=10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.003))
])
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.3)
deep_ffn.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# wide feedforward neural network
wide_ffn = models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(units=512, activation='relu'),
    # output layer: number of units is the number of classes
    tf.keras.layers.Dense(units=10, activation='softmax')
])
wide_ffn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
deep_ffn.summary()

In [ ]:
deep_hist = train_model(deep_ffn)

In [ ]:
plot_history(deep_hist)

In [ ]:
wide_ffn.summary()

In [ ]:
wide_hist = train_model(wide_ffn)

In [ ]:
plot_history(wide_hist)

We observe (informally):
 - neither the wide nor the deep network improve the classification performance significantly
 - the wide network has significantly more parameters
 - the deep network is slower than the wide network (only true on my machine)

## CNN

Next we implement a simple CNN by replacing the hidden Dense layer with a 2D convolutional layer.
The model's architecture looks as follows:

1. Input Layer: 2D images represented as 28x28 matrix
2. 2D Convolutional Layer with 32 independent 5x5 pixel filters
3. 2D MaxPooling Layer: summarizes pixel of a 2x2 neighborhood with their maximum. This aims to emphasize the most dominant feature.
4. Flatten Layer: Transforms the convolution output (12x12x32 tensor) into a 1D column vector.
5. Output layer, fully connected with 10 neurons and Softmax activation. Models a probability distribution over the available digits 0-9.

When applied on image data, CNNs have a various number of benefits over FFN:
- they preserve the spatial relations in the image
- due to parameter sharing of the kernels, patterns are detected regardless of the position in the image (translational invariance)
- they use fewer parameters than fully-connected layers and are less prone to overfitting

In [ ]:
simple_cnn = models.Sequential([
    layers.Conv2D(32, (5, 5), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(units=10, activation="softmax"),
])
simple_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
simple_cnn.summary()

In [ ]:
simple_cnn_hist = train_model(simple_cnn)

In [ ]:
plot_history(simple_cnn_hist)

We observe overfitting after roughly 7 epochs of training.

### High-Performance CNN

We implement a CNN with Keras according to the architecture proposed by Brendan Artley: https://medium.com/@BrendanArtley/mnist-keras-simple-cnn-99-6-731b624aee7f

In [ ]:
hp_cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same", input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(2, 2), strides=2),
    layers.Dropout(0.25),

    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(2, 2), strides=2),
    layers.Dropout(0.25),

    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(2, 2), strides=2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(1024, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
hp_cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hp_cnn.summary()

In [ ]:
hp_cnn_hist = train_model(hp_cnn)

In [ ]:
plot_history(hp_cnn_hist)

## Evaluation

In [ ]:
# methods to evaluate the model and visualize misclassifications

def evaluate_model(model):
    """Evaluate the model on the test set."""

    loss, accuracy = model.evaluate(x_test, y_test, batch_size=1024)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy*100:.2f}%")

    # Return the loss and accuracy
    return loss, accuracy

def find_misclassifications(model, n=20):
    predictions = model.predict(x_test, batch_size=1024)
    y_true = y_test.argmax(axis=1)
    y_pred = predictions.argmax(axis=1)

    # Get the indices corresponding to misclassifications
    misclassified_indices = np.where(y_pred != y_true)[0]

    for i, idx in enumerate(misclassified_indices):
        if i < n:
            visualize_miss(x_test[idx], y_true[idx], predictions[idx])
        else:
            break


def visualize_miss(x, y, y_pred):
    """Visualizes a single misclassification.
    """

    plt.figure(figsize=(12,1.5))

    # show the image and the predicted and true label in the lefthand subplot
    plt.subplot(1, 2, 1)
    plt.imshow(x, cmap='gray')
    plt.xlabel(f'Predicted: {np.argmax(y_pred)} Expected: {y}')

    # show the probabilities bar chart in the righthand subplot
    plt.subplot(1, 2, 2)
    plt.bar(range(10), y_pred * 100)
    plt.xticks(range(10))
    plt.xlabel('Digit')
    plt.ylabel('Probability [%]')
    plt.show()

In [ ]:
_ = evaluate_model(simple_ffn)
_ = evaluate_model(simple_cnn)
_ = evaluate_model(hp_cnn)

In [ ]:
find_misclassifications(hp_cnn, n=10)